# Deep Learning
## Assignment 3
Previously in 2_fullyconnected.ipynb, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [14]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in 1_notmnist.ipynb.

In [15]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f , encoding="latin1")
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (8000, 28, 28) (8000,)
Validation set (1000, 28, 28) (1000,)
Test set (1000, 28, 28) (1000,)


Reformat into a shape that's more adapted to the models we're going to train:

data as a flat matrix,
labels as float 1-hot encodings.

In [16]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (8000, 784) (8000, 10)
Validation set (1000, 784) (1000, 10)
Test set (1000, 784) (1000, 10)


In [17]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

# Problem 1
Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor t using nn.l2_loss(t). The right amount of regularization should improve your validation / test accuracy.

# Utilizando clasificador logístico.

In [18]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 10000
Beta = 0.001
graph = tf.Graph()
with graph.as_default():

  # Input data.
  # Load the training, validation and test data into constants that are
  # attached to the graph.
  tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
  tf_train_labels = tf.constant(train_labels[:train_subset])
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  # These are the parameters that we are going to be training. The weight
  # matrix will be initialized using random values following a (truncated)
  # normal distribution. The biases get initialized to zero.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  # We multiply the inputs with the weight matrix, and add biases. We compute
  # the softmax and cross-entropy (it's one operation in TensorFlow, because
  # it's very common, and it can be optimized). We take the average of this
  # cross-entropy across all training examples: that's our loss.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  loss = tf.reduce_mean( loss + Beta* tf.nn.l2_loss( weights ) )
  
  # Optimizer.
  # We are going to find the minimum of this loss using gradient descent.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  # These are not part of training, but merely here so that we can report
  # accuracy figures as we train.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [19]:
num_steps = 3001

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

with tf.Session(graph=graph) as session:
  # This is a one-time operation which ensures the parameters get initialized as
  # we described in the graph: random weights for the matrix, zeros for the
  # biases. 
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    # Run the computations. We tell .run() that we want to run the optimizer,
    # and get the loss value and the training predictions returned as numpy
    # arrays.
    _, l, predictions = session.run([optimizer, loss, train_prediction])
    if (step % 100 == 0):
      print('Loss at step %d: %f' % (step, l))
      print('Training accuracy: %.1f%%' % accuracy(
        predictions, train_labels[:train_subset, :]))
      # Calling .eval() on valid_prediction is basically like calling run(), but
      # just to get that one numpy array. Note that it recomputes all its graph
      # dependencies.
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 17.580593
Training accuracy: 13.1%
Validation accuracy: 18.5%
Loss at step 100: 4.720556
Training accuracy: 72.6%
Validation accuracy: 72.5%
Loss at step 200: 3.887126
Training accuracy: 75.4%
Validation accuracy: 74.8%
Loss at step 300: 3.326705
Training accuracy: 77.0%
Validation accuracy: 75.3%
Loss at step 400: 2.896517
Training accuracy: 78.0%
Validation accuracy: 75.8%
Loss at step 500: 2.550279
Training accuracy: 79.2%
Validation accuracy: 76.1%
Loss at step 600: 2.264815
Training accuracy: 79.9%
Validation accuracy: 76.9%
Loss at step 700: 2.025943
Training accuracy: 81.0%
Validation accuracy: 77.0%
Loss at step 800: 1.824214
Training accuracy: 81.8%
Validation accuracy: 77.6%
Loss at step 900: 1.652757
Training accuracy: 82.5%
Validation accuracy: 78.0%
Loss at step 1000: 1.506309
Training accuracy: 83.2%
Validation accuracy: 78.7%
Loss at step 1100: 1.380674
Training accuracy: 83.9%
Validation accuracy: 79.2%
Loss at step 1200: 1.272458
Training ac

# Utilizando una red neuronal de 2 capas

In [20]:
batch_size = 128
Beta = 0.001

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
    
  # new hidden layer
  hidden_nodes = 1024
  hidden_weights = tf.Variable( tf.truncated_normal([image_size * image_size, hidden_nodes]) )
  hidden_biases = tf.Variable( tf.zeros([hidden_nodes]))
  hidden_layer = tf.nn.relu( tf.matmul( tf_train_dataset, hidden_weights) + hidden_biases)
  
  # Variables.
  weights = tf.Variable( tf.truncated_normal([hidden_nodes, num_labels])) 
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(hidden_layer, weights) + biases
  loss = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(logits = logits, labels = tf_train_labels) )
  loss = tf.reduce_mean( loss + Beta *(tf.nn.l2_loss( weights) +  tf.nn.l2_loss(hidden_weights) ) )
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_relu = tf.nn.relu(  tf.matmul(tf_valid_dataset, hidden_weights) + hidden_biases)
  valid_prediction = tf.nn.softmax( tf.matmul(valid_relu, weights) + biases) 

  test_relu = tf.nn.relu( tf.matmul( tf_test_dataset, hidden_weights) + hidden_biases)
  test_prediction = tf.nn.softmax(tf.matmul(test_relu, weights) + biases)

In [21]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 100 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
      print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 603.703247
Minibatch accuracy: 10.9%
Validation accuracy: 42.1%
Test accuracy: 48.2%
Minibatch loss at step 100: 299.663208
Minibatch accuracy: 83.6%
Validation accuracy: 77.2%
Test accuracy: 84.0%
Minibatch loss at step 200: 258.917725
Minibatch accuracy: 91.4%
Validation accuracy: 80.0%
Test accuracy: 85.9%
Minibatch loss at step 300: 234.578629
Minibatch accuracy: 91.4%
Validation accuracy: 81.4%
Test accuracy: 86.5%
Minibatch loss at step 400: 211.226608
Minibatch accuracy: 95.3%
Validation accuracy: 80.3%
Test accuracy: 87.5%
Minibatch loss at step 500: 189.986435
Minibatch accuracy: 96.1%
Validation accuracy: 80.9%
Test accuracy: 87.8%
Minibatch loss at step 600: 171.530365
Minibatch accuracy: 95.3%
Validation accuracy: 80.7%
Test accuracy: 87.6%
Minibatch loss at step 700: 155.593063
Minibatch accuracy: 95.3%
Validation accuracy: 81.7%
Test accuracy: 88.2%
Minibatch loss at step 800: 140.279083
Minibatch accuracy: 100.0%
Validation accuracy:

# Problem 2
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

In [47]:
batch_size = 50
Beta = 0.1

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
    
  # new hidden layer
  hidden_nodes = 1024
  hidden_weights = tf.Variable( tf.truncated_normal([image_size * image_size, hidden_nodes]) )
  hidden_biases = tf.Variable( tf.zeros([hidden_nodes]))
  hidden_layer = tf.nn.relu( tf.matmul( tf_train_dataset, hidden_weights) + hidden_biases)
  
  # Variables.
  weights = tf.Variable( tf.truncated_normal([hidden_nodes, num_labels])) 
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(hidden_layer, weights) + biases
  loss = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(logits = logits, labels = tf_train_labels) )
  loss = tf.reduce_mean( loss + Beta *( tf.nn.l2_loss(weights) + tf.nn.l2_loss(hidden_weights) ) )
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_relu = tf.nn.relu(  tf.matmul(tf_valid_dataset, hidden_weights) + hidden_biases)
  valid_prediction = tf.nn.softmax( tf.matmul(valid_relu, weights) + biases) 

  test_relu = tf.nn.relu( tf.matmul( tf_test_dataset, hidden_weights) + hidden_biases)
  test_prediction = tf.nn.softmax(tf.matmul(test_relu, weights) + biases)

In [48]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 100 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
      print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 31836.408203
Minibatch accuracy: 8.0%
Validation accuracy: 16.9%
Test accuracy: 18.5%
Minibatch loss at step 100: 2.729958
Minibatch accuracy: 72.0%
Validation accuracy: 78.3%
Test accuracy: 82.2%
Minibatch loss at step 200: 1.445936
Minibatch accuracy: 82.0%
Validation accuracy: 77.7%
Test accuracy: 83.5%
Minibatch loss at step 300: 1.522731
Minibatch accuracy: 76.0%
Validation accuracy: 75.9%
Test accuracy: 81.6%
Minibatch loss at step 400: 1.275355
Minibatch accuracy: 90.0%
Validation accuracy: 75.3%
Test accuracy: 80.4%
Minibatch loss at step 500: 1.423575
Minibatch accuracy: 78.0%
Validation accuracy: 78.4%
Test accuracy: 83.3%
Minibatch loss at step 600: 1.409191
Minibatch accuracy: 76.0%
Validation accuracy: 79.0%
Test accuracy: 83.9%
Minibatch loss at step 700: 1.642386
Minibatch accuracy: 70.0%
Validation accuracy: 71.9%
Test accuracy: 76.4%
Minibatch loss at step 800: 1.410506
Minibatch accuracy: 78.0%
Validation accuracy: 73.9%
Test accu

# Problem 3
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides nn.dropout() for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

In [71]:
batch_size = 50
Beta = 0.1

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  tf_keep_prob = tf.placeholder(tf.float32)

  # new hidden layer
  hidden_nodes = 1024
  hidden_weights = tf.Variable( tf.truncated_normal([image_size * image_size, hidden_nodes]) )
  hidden_biases = tf.Variable( tf.zeros([hidden_nodes]))
  hidden_layer = tf.nn.relu( tf.matmul( tf_train_dataset, hidden_weights) + hidden_biases)
  
  hidden_layer_drop = tf.nn.dropout(hidden_layer, tf_keep_prob)
    
  # Variables.
  weights = tf.Variable( tf.truncated_normal([hidden_nodes, num_labels])) 
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(hidden_layer_drop, weights) + biases
  loss = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(logits = logits, labels = tf_train_labels) )
  loss = tf.reduce_mean( loss + Beta *( tf.nn.l2_loss(weights) + tf.nn.l2_loss(hidden_weights) ) )
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_relu = tf.nn.relu(  tf.matmul(tf_valid_dataset, hidden_weights) + hidden_biases)
  valid_prediction = tf.nn.softmax( tf.matmul(valid_relu, weights) + biases) 

  test_relu = tf.nn.relu( tf.matmul( tf_test_dataset, hidden_weights) + hidden_biases)
  test_prediction = tf.nn.softmax(tf.matmul(test_relu, weights) + biases)

In [72]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = { tf_train_dataset : batch_data, tf_train_labels : batch_labels, tf_keep_prob : float(0.5) }
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 100 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
      print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 31938.425781
Minibatch accuracy: 16.0%
Validation accuracy: 31.9%
Test accuracy: 34.9%
Minibatch loss at step 100: 2.944678
Minibatch accuracy: 70.0%
Validation accuracy: 77.1%
Test accuracy: 82.8%
Minibatch loss at step 200: 1.606993
Minibatch accuracy: 72.0%
Validation accuracy: 80.0%
Test accuracy: 83.9%
Minibatch loss at step 300: 1.777689
Minibatch accuracy: 62.0%
Validation accuracy: 74.3%
Test accuracy: 77.9%
Minibatch loss at step 400: 1.397037
Minibatch accuracy: 82.0%
Validation accuracy: 76.7%
Test accuracy: 81.8%
Minibatch loss at step 500: 1.576803
Minibatch accuracy: 74.0%
Validation accuracy: 77.9%
Test accuracy: 83.4%
Minibatch loss at step 600: 1.600289
Minibatch accuracy: 74.0%
Validation accuracy: 77.7%
Test accuracy: 82.9%
Minibatch loss at step 700: 1.847990
Minibatch accuracy: 66.0%
Validation accuracy: 71.7%
Test accuracy: 76.7%
Minibatch loss at step 800: 1.575566
Minibatch accuracy: 78.0%
Validation accuracy: 74.7%
Test acc

# Problem 4
Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is 97.1%.

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

global_step = tf.Variable(0)  # count the number of steps taken.
learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)

## lote de imagenes de entrenamiento = 150
## aplicamos la función learning_rate para el optimizador del gradiente descendiente
## beta = 0.00001
## Red neuronal de tres capas escondidas que implementan RELU con 1024, 512 y 256 nodos respectivamente y la capa final que produce la salida del modelo
## máximo puntaje obtenido en test: 92.2%

In [106]:
batch_size = 150
Beta = 0.00001

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  tf_keep_prob = tf.placeholder(tf.float32)

  # new hidden layer
  hidden_nodes = 1024
  hidden_weights = tf.Variable( tf.truncated_normal([image_size * image_size, hidden_nodes] , stddev=math.sqrt(2.0/(image_size*image_size))) )
  hidden_biases = tf.Variable( tf.zeros([hidden_nodes]))
  hidden_layer = tf.nn.relu( tf.matmul( tf_train_dataset, hidden_weights) + hidden_biases)
 
  # add dropout on hidden layer
  hidden_layer_drop = tf.nn.dropout(hidden_layer, tf_keep_prob)
  
  # new hidden layer2
  hidden_nodes2 = 512
  hidden_weights2 = tf.Variable( tf.truncated_normal([hidden_nodes, hidden_nodes2] , stddev=math.sqrt(2.0/(hidden_nodes))) )
  hidden_biases2 = tf.Variable( tf.zeros([hidden_nodes2]))
  hidden_layer2 = tf.nn.relu( tf.matmul( hidden_layer_drop, hidden_weights2) + hidden_biases2)

  # add dropout on hidden layer 2
  hidden_layer2_drop = tf.nn.dropout(hidden_layer2, tf_keep_prob)

# new hidden layer3
  hidden_nodes3 = 256
  hidden_weights3 = tf.Variable( tf.truncated_normal([hidden_nodes2, hidden_nodes3] , stddev=math.sqrt(2.0/(hidden_nodes2))) )
  hidden_biases3 = tf.Variable( tf.zeros([hidden_nodes3]))
  hidden_layer3 = tf.nn.relu( tf.matmul( hidden_layer2_drop, hidden_weights3) + hidden_biases3)

  # add dropout on hidden layer 3
  hidden_layer3_drop = tf.nn.dropout(hidden_layer3, tf_keep_prob)
    
  # Variables.
  weights = tf.Variable( tf.truncated_normal([hidden_nodes3, num_labels],stddev=math.sqrt(2.0/(hidden_nodes3)))) 
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(hidden_layer3_drop, weights) + biases
  loss = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(logits = logits, labels = tf_train_labels) )
  loss = tf.reduce_mean( loss + Beta *( tf.nn.l2_loss(weights) + tf.nn.l2_loss(hidden_weights) 
                                       + tf.nn.l2_loss(hidden_weights2) + tf.nn.l2_loss(hidden_weights3)))
  
  # Optimizer.
  global_step = tf.Variable(0)
  learning_rate = tf.train.exponential_decay(learning_rate = 0.75, global_step = global_step , decay_rate= 0.90 , decay_steps  =1000000)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss = loss, global_step = global_step)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_relu = tf.nn.relu(  tf.matmul(tf_valid_dataset, hidden_weights) + hidden_biases)
  valid_relu2 = tf.nn.relu( tf.matmul(valid_relu, hidden_weights2) + hidden_biases2)
  valid_relu3 = tf.nn.relu( tf.matmul(valid_relu2, hidden_weights3) + hidden_biases3)
  valid_prediction = tf.nn.softmax( tf.matmul(valid_relu3, weights) + biases) 

  test_relu = tf.nn.relu( tf.matmul( tf_test_dataset, hidden_weights) + hidden_biases)
  test_relu2 = tf.nn.relu( tf.matmul( test_relu, hidden_weights2) + hidden_biases2)
  test_relu3 = tf.nn.relu( tf.matmul( test_relu2, hidden_weights3) + hidden_biases3)
  test_prediction = tf.nn.softmax(tf.matmul(test_relu3, weights) + biases)

In [107]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = { tf_train_dataset : batch_data, tf_train_labels : batch_labels, tf_keep_prob : float(0.5) }
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 100 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
      print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.099688
Minibatch accuracy: 9.3%
Validation accuracy: 21.6%
Test accuracy: 24.3%
Minibatch loss at step 100: 0.756204
Minibatch accuracy: 75.3%
Validation accuracy: 82.5%
Test accuracy: 87.6%
Minibatch loss at step 200: 0.842549
Minibatch accuracy: 74.0%
Validation accuracy: 81.7%
Test accuracy: 85.9%
Minibatch loss at step 300: 0.668393
Minibatch accuracy: 77.3%
Validation accuracy: 84.2%
Test accuracy: 89.4%
Minibatch loss at step 400: 0.530206
Minibatch accuracy: 82.0%
Validation accuracy: 84.1%
Test accuracy: 89.0%
Minibatch loss at step 500: 0.450817
Minibatch accuracy: 88.0%
Validation accuracy: 83.7%
Test accuracy: 89.7%
Minibatch loss at step 600: 0.556656
Minibatch accuracy: 84.7%
Validation accuracy: 83.7%
Test accuracy: 90.5%
Minibatch loss at step 700: 0.502620
Minibatch accuracy: 86.0%
Validation accuracy: 83.8%
Test accuracy: 90.3%
Minibatch loss at step 800: 0.376760
Minibatch accuracy: 89.3%
Validation accuracy: 84.6%
Test accuracy